In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
import os

dir_name = "PatientTrajectoryForecasting"
os.chdir(dir_name) if os.path.isdir(dir_name) else None

In [110]:
from utils.data_processing import format_data, prepare_sequences, filter_codes, train_test_val_split, get_optimal_embedding_size
from utils.utils import load_data, get_paths, store_files
from dataclasses import dataclass
import yaml

In [15]:
with open('paths.yaml', 'r') as file:
    path_config = yaml.safe_load(file)

In [29]:
@dataclass
class Config:
    strategy = 'SDP'
    predict_procedure : bool = False
    predict_drugs : bool = False
    procedure : bool = not(predict_procedure)
    drugs : bool = not(predict_drugs)
    truncate : bool = True
    pad : bool = True
    input_max_length :int = 448
    target_max_length :int = 64
    test_size : float = 0.05
    valid_size : float = 0.05
    
config = Config()

# This first part is for preparing the data

In [ ]:
train_data_path = get_paths(path_config, config.strategy, config.predict_procedure, config.predict_procedure, train = True)

In [ ]:
patients_visits_sequences, tokens_to_ids_map, ids_to_types_map = load_data(train = False)

In [ ]:
source_target_sequences = format_data(patients_visits_sequences, config.strategy)

In [ ]:
source_target_sequences, _ = filter_codes(source_target_sequences, ids_to_types_map, config.procedure, config.drugs,\
                                         reset_target_map = False )

In [ ]:
store_files(source_target_sequences = source_target_sequences,
            ids_to_types_map = ids_to_types_map, 
            tokens_to_ids_map = tokens_to_ids_map,
            ids_to_tokens_map = ids_to_tokens_map, 
            output_file = train_data_path['train_data_path'],
            train = True)

# Load data

## Next time start from here if using the same codes & strategy:

In [22]:
ls outputData/SDP/

Inp_d_p_dr_out_d/


In [18]:
train_data_path = get_paths(path_config, config.strategy, config.predict_procedure, config.predict_procedure, train = True)

True


In [24]:
source_target_sequences, ids_to_types_map, tokens_to_ids_map, ids_to_tokens_map = load_data(train_data_path['train_data_path'], train = True)

In [25]:
source_sequences, target_sequences = prepare_sequences(source_target_sequences, tokens_to_ids_map, config.truncate , config.pad,\
                            config.input_max_length, config.target_max_length)

In [82]:
data_properties = get_optimal_embedding_size(source_sequences, target_sequences)

must reformat, too much memory would be lost in embedding


In [105]:
train_data_path = get_paths(path_config, config.strategy, config.predict_procedure, config.predict_procedure, train = True, processed_data = True)

In [107]:
store_files(**data_properties, processed_data = True, output_file=train_data_path['processed_data_path'])

dumping in outputData/SDP/Inp_d_p_dr_out_d/processed_data/


In [111]:
source_sequences, target_sequences, _ , new_to_old_ids_target = load_data(train_data_path['processed_data_path'], processed_data = True)

new_to_old_ids_source file not availble, mapping is the same as the old on


In [27]:
train, test, val = train_test_val_split(source_sequences, target_sequences, test_size = 0.1, valid_size = 0.1)